<a href="https://colab.research.google.com/github/Hemachandran-D45/vectordb/blob/main/RAG_Pipeline_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install langchain_community -q
!pip install langchain_huggingface -q
!pip install huggingface_hub -q
!pip install langchain_core -q

In [ ]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("/content/RohitSharma.txt")
loader.load()

In [6]:
rohit_doc = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [52]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

In [20]:
documents =text_splitter.split_documents(rohit_doc)

In [ ]:
#for i, chunk in enumerate(documents):
 # print(f"Chunk {i+1} (Length: {len(chunk.page_content)} character)")
  #print(chunk.page_content)

In [28]:
!pip install chromadb -q
!pip install tiktoken -q

In [29]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
db = Chroma.from_documents(documents, HuggingFaceEmbeddings())

In [ ]:
!pip install transformers langchain_huggingface

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline


hf_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2" , max_new_tokens = 250)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [83]:
query= "Who was the sucessful captain in India?"
result = db.similarity_search(query)

In [85]:
response = hf_pipeline({
    "question": query,
    "context": result[0].page_content,
})
print(response['answer'])

Rohit Sharma
